In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import xarray as xr

from discontinuum.models.loadest_gp.providers import usgs
from discontinuum.utils import aggregate_to_daily

In [ ]:
# setup

# Choptank River at Greensboro, MD
site = "01491000" 
start_date = "1979-10-01"
end_date = "2011-09-30"

# SF Coeur D Alene River 
site = "12413470"
start_date = "1988-10-01" 
end_date = "2021-09-30" 

characteristic = 'Inorganic nitrogen (nitrate and nitrite)'
#characteristic = 'Phosphorus'
fraction = 'Dissolved'

In [ ]:
# download data

daily = usgs.get_daily(site=site, start_date=start_date, end_date=end_date)
samples = usgs.get_samples(site=site, start_date=start_date, end_date=end_date, characteristic=characteristic, fraction=fraction)

samples = aggregate_to_daily(samples)

training_data = xr.merge([samples, daily], join='inner')

In [ ]:
%%time
# fit model

from discontinuum.models import LoadestGP

model = LoadestGP()
model.fit(target=training_data['concentration'], covariates=training_data[['time','flow']])

In [ ]:
# plot result

model.plot(daily[['time','flow']])

In [ ]:
#  plot
model.contourf(cmap='magma', levels=5)
plt.show()

In [ ]:
sim = model.sample(daily[['time','flow']])

In [ ]:
flux = sim * daily['flow'] * 86400 * 1e-6 #kg/day
flux.attrs = samples['concentration'].attrs
flux.attrs['units'] = 'kilotons N per year'

In [ ]:
#compute load then do
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
annual_flux = flux.resample(time='YE').sum()

# plot
annual_flux.to_dataframe(name='load').boxplot(by='time', showfliers=False, grid=False, ax=ax)
ax.set_ylabel('{}\n[{}]'.format(flux.attrs['long_name'], flux.attrs['units']))
ax.set_xlabel('Year')
ax.tick_params(axis='x', labelrotation=90)

# label every 5th year
labels = ['' if (year % 5 != 0) else year for i, year in enumerate(annual_flux.time.dt.year.values)]
ax.set_xticklabels(labels)
ax.set_title('')

plt.show()

In [ ]:
fig, ax = plt.subplots()

flux_5yr = annual_flux.rolling(time=5).mean()
flux_5yr['time'] = flux_5yr.time.dt.year
flux_5yr.to_dataframe(name='load').boxplot(by='time', showfliers=False, grid=False, ax=ax)

ax.set_ylabel('{}\n[{}]'.format(flux.attrs['long_name'], flux.attrs['units']))
ax.set_xlabel('Year')
ax.tick_params(axis='x', labelrotation=90)

# label every 5th year
labels = ['' if (year % 5 != 0) else year for i, year in enumerate(annual_flux.time.dt.year.values)]
ax.set_xticklabels(labels)
ax.set_title('')


plt.show()

In [ ]:
flow_5yr = daily['flow'].resample(time='YE').sum().rolling(time=5).mean()
flow_5yr['time'] = flow_5yr.time.dt.year
flow_5yr.plot()